### Vertebral extraction using TotalSegmentator and summarises the areas of all of the vertebrae at each slice

#### By Wei Hong and Tianpei Yang

In [ ]:
import os
import nibabel as nib
import numpy as np
from totalsegmentator.python_api import totalsegmentator
import pandas as pd
import torch
from pathlib import Path

if torch.cuda.is_available():
    print("GPU: detected")
else:
    print("GPU: not detected")

In [ ]:
TS_vertebrae = ["vertebrae_C7", "vertebrae_T1", "vertebrae_T2", "vertebrae_T3", "vertebrae_T4", "vertebrae_T5",
                "vertebrae_T6", "vertebrae_T7", "vertebrae_T8", "vertebrae_T9", "vertebrae_T10", "vertebrae_T11",
                "vertebrae_T12", "vertebrae_L1", "vertebrae_L2", "vertebrae_L3", "vertebrae_L4", "vertebrae_L5",
                "vertebrae_S1"]

##### Note: the code segments below assumes that the CTs are stored under the file structure /input_root/patient/series/subseries/.dcms

"patient" - unique identifier of each patient eg) UR

"series" - unique identifier of each CT series, at least 5 digits

"subseries" - unique identifier of each subseries under each series, at least 5 digits

".dcms" - the dicom files under each subseries

The whole workflow is designed to be left unattended, automatically skipping invalid scans and scans that have already been processed, as well as moving on if some other error is encountered.

In [ ]:
def TSlumbosacral_segment(input_root, patient, series, subseries, output_root):
    """
    Processes a DICOM subseries for lumbosacral segmentation using TotalSegmentator.
    Inputs a scan and outputs the segmentation masks of all the vertebrae of interest.

    This function takes a specific DICOM subseries, performs checks for validity
    and prior processing, and then attempts to segment lumbosacral vertebrae
    using the 'totalsegmentator' tool. 

    Args:
        input_root (str): The root directory for input DICOM data.
        patient (str): The patient identifier (subdirectory under /input_root).
        series (str): The series identifier (subdirectory under /input_root/patient).
        subseries (str): The subseries identifier (subdirectory under /input_root/patient/series).
        output_root (str): The root directory for output
    """

    input_path = os.path.join(input_root, patient, series, subseries)
    output_path = os.path.join(output_root, patient, series[-5:], subseries[-5:])
    dcm_count = sum(1 for f in os.listdir(input_path) if f.endswith(".dcm"))
    if dcm_count < 10:
        print("- Invalid subseries")
        return
    os.makedirs(output_path, exist_ok = True)
    output_count = sum(1 for f in os.listdir(output_path) if f.endswith(".nii.gz"))
    if output_count == 19:
        print("- Subseries already done")
        return
    try:
        totalsegmentator(input_path, output_path, roi_subset = TS_vertebrae)
    except Exception as e:
        print("- Error processing", patient, series, subseries, str(e))
        # save the error message
        error_path = os.path.join(output_path, str(e))
        with open(error_path, "w") as file:
            pass

def TSlumbosacral_segment_batch(root_dir, output_dir):
    """
    Batches the lumbosacral segmentation process for all patient, series,
    and subseries directories found under a given root directory.

    This function recursively traverses a directory structure (root_dir/patient/series/subseries),
    calls `TSlumbosacral_segment` for each valid subseries, and provides
    basic logging and unhandled exception capture.

    Args:
        root_dir (str): The root directory containing patient data, structured as
                        root_dir/patient_id/series_id/subseries_id
        output_dir (str): The root directory where segmentation results will be saved.
    """
    for patient_dir in os.listdir(root_dir):
        patient_path = os.path.join(root_dir, patient_dir)
        if os.path.isdir(patient_path):
            for series_dir in os.listdir(patient_path):
                series_path = os.path.join(patient_path, series_dir)
                if os.path.isdir(series_path):  
                    for subseries_dir in os.listdir(series_path):
                        subseries_path = os.path.join(series_path, subseries_dir)
                        if os.path.isdir(subseries_path):
                            print("Processing:", subseries_path)
                            try:
                                TSlumbosacral_segment(root_dir, patient_dir, series_dir, subseries_dir, output_dir)
                            except Exception as e: # for errors not caught within TSlumbosacral_segment()
                                print("- Unhandled exception processing", patient_dir, series_dir, subseries_dir)
                                error_path = os.path.join(output_dir, patient_dir, series_dir[-5:], subseries_dir[-5:], "UNHANDLED_EXCEPTION")
                                with open(error_path, "w") as file:
                                    pass
                               
def TSlumbosacral_count(input_root, patient, series, subseries, output_root):
    """
    Calculates and saves the cross-sectional area (pixel count) of each vertebral
    slice within a lumbosacral segmentation, and records slice distances.

    This function expects pre-segmented mask files for lumbosacral vertebrae 
    from 'TSlumbosacral_segment' or 'TSlumbosacral_segment_batch'
    It computes the pixel count for each slice of each vertebra and stores
    this data, along with scaled craniocaudal displacement, into an Excel file.

    Args:
        input_root (str): The root directory of the original patient data. 
        While not directly used for loading NIfTI files within this function, 
        it is used to define the full input path for context (and consistency with batching)
        patient (str): The patient identifier (subdirectory under /input_root).
        series (str): The series identifier (subdirectory under /input_root/patient).
        subseries (str): The subseries identifier (subdirectory under /input_root/patient/series).
        output_root (str): The root directory for output
    """
    input_path = os.path.join(input_root, patient, series, subseries)
    output_path = os.path.join(output_root, patient, series[-5:], subseries[-5:])
    output_filename = os.path.join(output_root, patient + ";" + series + ";" + subseries + ".xlsx")
    if os.path.exists(output_filename):
        print("- Subseries already done")
        return
    output = pd.DataFrame()
    # slice distance
    for vertebra0 in [TS_vertebrae[0]]:
        nifti_file = nib.load(output_path + "/" + vertebra0 + ".nii.gz")
        affine = nifti_file.affine
        slice_distance = np.linalg.norm(affine[:3, 2])
        # print("Slice distance:", slice_distance)
    # slice areas
    for vertebra in TS_vertebrae:
        vertebra_filename = vertebra + ".nii.gz"
        vertebra_path = os.path.join(output_path, vertebra_filename)
        # load vertebra masks in reverse order (mask output is caudocranial, original CT is craniocaudal)
        vertebra_masks = nib.load(vertebra_path).get_fdata()[:, :, ::-1]
        pixel_counts = []
        for slice_idx in range(vertebra_masks.shape[2]):
            slice_data = vertebra_masks[:, :, slice_idx]
            nonzero_count = np.count_nonzero(slice_data)
            pixel_counts.append(nonzero_count)
        # print(vertebra, pixel_counts, len(pixel_counts))
        output[vertebra[10:]] = pixel_counts
    indices = list(output.index)
    scaled_indices = [i * slice_distance for i in indices]
    output["x"] = scaled_indices #* slice_thickness # x is the craniocaudal displacement
    output.to_excel(os.path.join(output_root, patient + ";" + series + ";" + subseries + ".xlsx"), index = False)

def TSlumbosacral_count_batch(root_dir, output_dir):
    """
    Batches the lumbosacral vertebral slice counting process for all
    patient, series, and subseries directories found under a given root directory.

    This function recursively traverses a directory structure (patient/series/subseries),
    calls `TSlumbosacral_count` for each valid subseries, and provides
    basic logging and unhandled exception capture. It assumes that the
    segmentation files are already available post 'TSlumbosacral_segment' or 'TSlumbosacral_segment_batch'

    Args:
        root_dir (str): The root directory containing patient data (e.g., original DICOMs).
                        This is passed as `input_root` to `TSlumbosacral_count`.
        output_dir (str): The root directory where segmented NIfTI files are expected
                          and where the final Excel output files will be saved.
    """
    for patient_dir in os.listdir(root_dir):
        patient_path = os.path.join(root_dir, patient_dir)
        if os.path.isdir(patient_path):
            for series_dir in os.listdir(patient_path):
                series_path = os.path.join(patient_path, series_dir)
                if os.path.isdir(series_path):  
                    for subseries_dir in os.listdir(series_path):
                        subseries_path = os.path.join(series_path, subseries_dir)
                        if os.path.isdir(subseries_path):
                            print("Processing:", subseries_path)
                            try:
                                TSlumbosacral_count(root_dir, patient_dir, series_dir, subseries_dir, output_dir)
                            except Exception as e:
                                print("- Unhandled exception processing", patient_dir, series_dir, subseries_dir)

In [ ]:
# PART 1: segment all subseries, skipping if insufficient .dcm files or if already processed, with masks (or error messages) saved under the same folder structure as the input
TSlumbosacral_segment_batch("D:\input_path", "F:\output_path")

# PART 2: count pixels for all subseries and save as .xlsx
TSlumbosacral_count_batch("D:\input_path", "F:\output_path")